## Shapley effects for RC and $\theta_{11}$

In [17]:
import os
import numpy as np
import matplotlib.pyplot as plt
from ruspy.estimation.estimation import estimate
from ruspy.model_code.demand_function import get_demand 
from python.econsa_shapley import get_shapley
from python.econsa_shapley import _r_condmvn
from python.sampling import get_cov_and_mean_rc_theta_32
#from python.conditional_sampling import x_all
#from python.conditional_sampling import x_cond

In [3]:
# Set simulating variables.
disc_fac = 0.9999
num_buses = 50
num_periods = 120
gridsize = 1000
# We use the cost parameters and transition probabilities from the replication.
params = np.array([10.07780762, 2.29417622])
trans_probs = np.array([0.39189182, 0.59529371, 0.01281447])
scale = 1e-3

init_dict_simulation = {
    "simulation": {
        "discount_factor": disc_fac,
        "periods": num_periods,
        "seed": 123,
        "buses": num_buses,
    },
    "plot": {"gridsize": gridsize},
}

In [5]:
# Calcualte objects necessary for the simulation process. See documentation for details.
num_states = 200
costs = calc_obs_costs(num_states, lin_cost, params, scale)

trans_mat = create_transition_matrix(num_states, trans_probs)
ev = calc_fixp(trans_mat, costs, disc_fac)[0]

In [6]:
# Use one init_dict for get_demand() and estimate().
init_dict_estimation = {
    'model_specifications': {
        'discount_factor': disc_fac,
        'number_states': num_states,
        'maint_cost_func': 'linear',
        'cost_scale': 1e-3
    },
    'optimizer': {
        'approach': 'NFXP',
        'algorithm': 'scipy_L-BFGS-B',
        'gradient': 'Yes'
    },
    
}

In [8]:
num_sim = 200
cov, mean = get_cov_and_mean_rc_theta_32(num_sim,
                                         ev, 
                                         costs, 
                                         trans_mat, 
                                         init_dict_simulation, 
                                         init_dict_estimation,
                                        )
np.save(os.path.join('data', 'cov_rc_theta_11'), cov)
np.save(os.path.join('data', 'mean_rc_theta_11'), mean)

In [80]:
cov = np.load(os.path.join('data', 'cov_rc_theta_11.npy'))
mean = np.load(os.path.join('data', 'mean_rc_theta_11.npy'))

Define functions `rust_model()`, and function arguments.

In [75]:
def x_all(n):
    distribution = cp.MvNormal(mean, cov)
    return distribution.sample(n)

def x_cond(n, subset_j, subsetj_conditional, xjc):
    if subsetj_conditional is None:
        cov_int = np.array(cov)
        cov_int = cov_int.take(subset_j, axis=1)
        cov_int = cov_int[subset_j]
        distribution = cp.MvNormal(mean[subset_j], cov_int)
        return distribution.sample(n)
    else:
        return _r_condmvn(
            n,
            mean=mean,
            cov=cov,
            dependent_ind=subset_j,
            given_ind=subsetj_conditional,
            x_given=xjc,
        )

In [95]:
def rust_model(x):
    if method == 'exact':
        n_evaluations = n_output + np.math.factorial(n_inputs) * (n_inputs -1) * n_outer * n_inner
    elif method == 'random':
        n_evaluations = n_output + n_perms * (n_inputs -1) * n_outer * n_inner
    demand_inputs = np.zeros((n_evaluations, 5))
    demand_inputs[:, :3] = np.array([0.39189182, 0.59529371, 0.01281447])
    demand_inputs[:, 3:] = x[:, :]
    
    demand_output = np.zeros((n_evaluations, 1))
    
    for sample in np.arange(n_evaluations):
        demand_params = demand_inputs[sample, :]
        demand_output[sample] = get_demand(init_dict_estimation, demand_dict, demand_params).iloc[0][0]
        
    return demand_output

In [98]:
# Need demand at certain value of RC only.
demand_dict = {
    "RC_lower_bound": 11.5,
    "RC_upper_bound": 11.5,
    "demand_evaluations": 1,
    "tolerance": 1e-10,
    "num_periods": num_periods,
    "num_buses": num_buses,
}

In [96]:
n_inputs = 2
method = 'exact'
np.random.seed(1234)

if method == 'exact':
    n_perms = None
elif method == 'random':
    n_perms = 10
n_output = 10**3
n_outer = 15
n_inner = 3

In [99]:
%%time
exact_shapley = get_shapley(method, rust_model, x_all, x_cond, n_perms, n_inputs, n_output, n_outer, n_inner)
exact_shapley.rename(index={'X1': 'RC', 'X2': 'theta_11'}, inplace=True)

Wall time: 6min 47s


In [106]:
exact_shapley

,Shapley effects,std. errors,CI_min,CI_max
RC,0.449098,0.31756,-0.173320,1.071515
theta_11,0.550902,0.31756,-0.071515,1.173320
